## Amazon Web Scraper
Requirements

* Selenium
* BeautifulSoup

In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
#!pip install selenium
#!pip install beautifulsoup4

In [3]:
from selenium import webdriver

In [4]:
driver = webdriver.Chrome()

In [5]:
def get_url(search_term):
    """ Generate a URL from search term"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    
    # add term to query url
    url = template.format(search_term)
    
    # add page query place holder
    url += '&page{}'
    
    return url

def extract_record(item):
    """ Extract and return data from a single record"""
    #description and URL
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        #Price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    try:
        #Rating and Review
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'})
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    record = []
    url = get_url(search_term)
    
    records = []
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
            
    driver.close()
    
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Review Count', 'URL'])
        writer.writerows(records)

In [6]:
main('shoes')